In [2]:
import os
import rasterio
from rasterio.windows import Window

In [5]:
print("Current Working Directory " , os.getcwd())

Current Working Directory  c:\Users\laris\repos\eo-data


In [4]:
# this will move one directory up
os.chdir('..')

In [18]:
# paths to the raster files 

tif1009 = 'data/rs/20221009_042654_SN21_L1_SR_MS_80157/rasters/20221009_042654_SN21_L1_SR_MS_0.tif'
tif1016 = 'data/rs/20221016_043447_SN22_L1_SR_MS_82727/rasters/20221016_043447_SN22_L1_SR_MS_0.tif'
tif1019 = 'data/rs/20221019_001540_SN16_L1_SR_MS_82727/rasters/20221019_001540_SN16_L1_SR_MS_0.tif'

In [19]:
dataset = rasterio.open(tif1016)

In [20]:
# counts how many bands are in the loaded raster
dataset.count

4

##Calculating Indices

In [21]:
# TRVI 
# Open the TIFF file
with rasterio.open(tif1016) as src:
    # read and save as variable metadata of source raster
    src_meta = src.profile

    # Create windows for each band
    nir_window = Window(col_off=0, row_off=0, width=src.width, height=src.height)
    red_window = nir_window
    green_window = nir_window
    blue_window = nir_window

    # Read the specific bands as numpy arrays
    nir_band = src.read(4, window=nir_window)
    red_band = src.read(3, window=red_window)
    green_band = src.read(2, window=green_window)
    blue_band = src.read(1, window=blue_window)
    
    # Perform the calculations of indexes - reference for some https://towardsdatascience.com/monitor-vegetation-with-google-earth-engine-909a2ad51a48
    trvi = 4 * ((nir_band - red_band) / (nir_band + red_band + green_band + blue_band))
    # 4 * ((B4-B3)/(B4+B3+B2+B1))
    ndvi = (nir_band - red_band) / (nir_band + red_band)
    evi =  2.5 * ((nir_band - red_band) / (nir_band + 6*red_band - 7.5*blue_band + 1))

    print(trvi)
    print(ndvi)
    print(evi)
   
    print(trvi.shape)
    print(src_meta)

C:\Users\laris\AppData\Local\Temp\ipykernel_29388\3308843451.py:20: RuntimeWarning: invalid value encountered in divide
  trvi = 4 * ((nir_band - red_band) / (nir_band + red_band + green_band + blue_band))
C:\Users\laris\AppData\Local\Temp\ipykernel_29388\3308843451.py:21: RuntimeWarning: invalid value encountered in divide
  ndvi = (nir_band - red_band) / (nir_band + red_band)
C:\Users\laris\AppData\Local\Temp\ipykernel_29388\3308843451.py:22: RuntimeWarning: divide by zero encountered in divide
  evi =  2.5 * ((nir_band - red_band) / (nir_band + 6*red_band - 7.5*blue_band + 1))


[[       nan        nan        nan ... 0.72477805 0.74078443 0.77320247]
 [       nan        nan        nan ... 0.76538309 0.72478429 0.78606965]
 [       nan        nan        nan ... 0.78761524 0.69109948        nan]
 ...
 [0.24658494 0.27913858 0.11082379 ...        nan        nan        nan]
 [0.22494314 0.32225084 0.14809616 ...        nan        nan        nan]
 [0.23107918 0.35015432 0.14522048 ...        nan        nan        nan]]
[[       nan        nan        nan ... 0.32232592 0.33052632 0.34162896]
 [       nan        nan        nan ... 0.33872101 0.32137203 0.3459854 ]
 [       nan        nan        nan ... 0.34428224 0.30877193        nan]
 ...
 [0.12028901 0.1364513  0.05266854 ...        nan        nan        nan]
 [0.10968473 0.15908913 0.07145276 ...        nan        nan        nan]
 [0.11266806 0.1743373  0.07066711 ...        nan        nan        nan]]
[[ 0.          0.          0.         ... -3.18214033 -2.10174029
  -2.22331288]
 [ 0.          0.          0.  

In [22]:
# saving calculations as new geotif

# create output directory if not exists

if os.path.exists('data/rs/outputs'):
    print('The directory exists!')
else:
    os.makedirs('data/rs/outputs')



The directory exists!


In [23]:
# writing calculated index out as tif  20221009 - date

# export trvi index as a new geotiff to use or share with colleagues
with rasterio.open('data/rs/outputs/20221016_trvi.tiff', 'w', **src_meta) as ff:
    ff.write(trvi,1)
with rasterio.open('data/rs/outputs/20221016_ndvi.tiff', 'w', **src_meta) as ff:
    ff.write(ndvi,1)
with rasterio.open('data/rs/outputs/20221016_evi.tiff', 'w', **src_meta) as ff:
    ff.write(evi,1)

c:\Users\laris\repos\.venv\lib\site-packages\numpy\core\_asarray.py:130: RuntimeWarning: invalid value encountered in cast
  arr = array(a, dtype=dtype, order=order, copy=False, subok=subok)


In [15]:
# Register GDAL format drivers and configuration options with a
# context manager.
with rasterio.Env():

    # Write an array as a raster band to a new 8-bit file. For
    # the new file's profile, we start with the profile of the source
    profile = src.profile

    # And then change the band count to 1, set the
    # dtype to uint8, and specify LZW compression.
    profile.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open('trvi.tif', 'w', **profile) as dst:
        dst.write(array.astype(rasterio.uint8), 1)

# At the end of the ``with rasterio.Env()`` block, context
# manager exits and all drivers are de-registered.

RasterioIOError: Dataset is closed: data/rs/20221009_042654_SN21_L1_SR_MS_80157/rasters/20221009_042654_SN21_L1_SR_MS_0.tif